In [1]:
import pandas as pd
import psycopg2

def fetch_query_records(query, columns):
    """
    Creates a connection to database, returns query from specified table.

    Input: query: a SQL query (string)

    Returns: response: cursos.fetchall() object in array form
    """
    DB_USER="citrics"
    DB_PASSWORD="BnDW2WupbFpgZSewsZm7"
    DB_NAME="postgres"
    DB_HOST="citricsads.cav8gkdxva9e.us-east-1.rds.amazonaws.com"

    # Creating Connection Object
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST)
    # Creating Cursor Object
    cursor = conn.cursor()
    # Fetch query
    query = query
    # Execute query
    cursor.execute(query)
    # Query results
    response = list(cursor.fetchall())
    # Closing Connection
    conn.close()

    return response

def fetch_query(query, columns):
    """
    Creates a connection to database, returns query from specified table
    as a list of dictionaries.

    Input: query: a SQL query (string)

    Returns: pairs: dataframe of cursor.fetchall() response in JSON pairs
    """
    
    # Fetch query
    response = fetch_query_records(query, columns)

    # Key-value pair names for df columns
    columns = columns
    # List of tuples to DF
    df = pd.DataFrame(response, columns=columns)
    # DF to dictionary
    pairs = df.to_json(orient='records')

    return pairs

# Goal: Compile `census`, `bls_jobs`, `walkability`, and `rental` into single, merged dataset
- Should align with front-end form where end user can enter preferences for:
    - Population
    - Average Rent
    - WalkScore
    - Primary Job Industry

### Start by creating df instances from each table in DB via SQL queries

In [8]:
### Rental Data

query = """
SELECT *
FROM rental
WHERE "month" = '2020-08-01'
"""

columns = ["month", "city", "state", "Studio", "onebr", "twobr", "threebr", "fourbr"]

rental = pd.read_json(fetch_query(query, columns))

rental = rental.drop("month", axis=1)

rental.columns = [x.lower() for x in rental.columns.to_list()]

print(rental.shape)
rental.head()

(444, 7)


,city,state,studio,onebr,twobr,threebr,fourbr
0,King of Prussia,PA,1038,1232,1488,1862,2072
1,Las Cruces,NM,514,599,726,1045,1280
2,North East,MD,909,1079,1303,1630,1814
3,St. Cloud,MN,578,712,883,1218,1543
4,Pasadena,CA,1350,1606,2064,2806,3093


In [107]:
### BLS Data

query = """
SELECT DISTINCT ON (j.city) j.*
FROM bls_jobs j
ORDER BY j.city, j.loc_quotient DESC
"""

columns = [
    "city",
    "state",
    "occ_title",
    "jobs_1000",
    "loc_quotient",
    "hourly_wage",
    "annual_wage"]

bls = pd.read_json(fetch_query(query, columns))

bls = bls.drop(["jobs_1000","loc_quotient"], axis=1)

bls = bls.loc[bls.annual_wage != 0]

print(bls.shape)
bls.head()

(362, 5)


,city,state,occ_title,hourly_wage,annual_wage
0,Abilene,TX,Wellhead Pumpers,30.14,62680
1,Aguadilla-Isabela,PR,Telemarketers,8.16,16980
2,Akron,OH,"Patternmakers, Metal and Plastic",17.27,35930
3,Albany,OR,Fallers,35.98,74850
4,Albany-Schenectady-Troy,NY,"Physical Scientists, All Other",46.86,97470


In [21]:
### WalkScores

query = """
SELECT *
FROM WALKABILITY
"""

columns = ["city", "walkscore"]

walkscores = pd.read_json(fetch_query(query, columns))

walkscores["state"] = [x[-2:] for x in walkscores.city.to_list()]
walkscores["city"] = [x.split(",")[0] for x in walkscores.city.to_list()]

walkscores = walkscores[["city","state","walkscore"]]

print(walkscores.shape)
walkscores.head()

(444, 3)


,city,state,walkscore
0,Houston,TX,86.28
1,Philadelphia,PA,97.70
2,Phoenix,AZ,71.76
3,San Antonio,TX,65.46
4,San Diego,CA,80.94


In [40]:
### Census Data

query = """
SELECT DISTINCT ON (j.city)
	j.city,
	j. "state",
	j.popestimate2019
FROM
	census j
ORDER BY
	j.city,
	j.popestimate2019 DESC
"""

# Re-factored query to reduce num of duplicates

columns = [
    "city",
    "state",
    "popestimate2019"]

census = pd.read_json(fetch_query(query, columns))

print(census.shape)
census.head()

(31802, 3)


,city,state,popestimate2019
0,Aastad township,MN,213
1,Abbeville city,LA,12038
2,Abbeville County,SC,24527
3,Abbeville town,MS,423
4,Abbotsford city,WI,2233


In [37]:
census.loc[census.city == "Baltimore city"]

# Problem with census data is the term 'city' after city names ...
# Also has several duplicates ...

,city,state,popestimate2019
27970,Baltimore city,MD,593490
28328,Baltimore city,MD,593490
28329,Baltimore city,MD,593490


In [42]:
census.city[1].replace(" city","")

'Abbeville'

In [43]:
census.city = [x.replace(" city","") for x in census.city.to_list()]

print(census.shape)
census.head()

(31802, 3)


,city,state,popestimate2019
0,Aastad township,MN,213
1,Abbeville,LA,12038
2,Abbeville County,SC,24527
3,Abbeville town,MS,423
4,Abbotsford,WI,2233


In [45]:
census.loc[(census.city == "Baltimore") & (census.state == "MD")]
# Now looks more appropriate

,city,state,popestimate2019
5455,Baltimore,MD,593490


# Inspect Data sets

In [108]:
print(walkscores.shape)
print(census.shape)
print(rental.shape)
print(bls.shape)

(444, 3)
(31802, 3)
(444, 7)
(362, 5)


In [109]:
print("Walkscore states:", walkscores.state.nunique())
print("Census states:", census.state.nunique())
print("Rental states:", rental.state.nunique())
print("BLS states:", bls.state.nunique())

Walkscore states: 46
Census states: 51
Rental states: 46
BLS states: 90


In [56]:
codes = [
    'AL',
    'AK',
    'AZ',
    'AR',
    'CA',
    'CO', 
    'CT',
    'DE', 
    'DC',
    'FL',
    'GA',
    'HI',
    'ID',
    'IL',
    'IN',
    'IA',
    'KS',
    'KY',
    'LA',
    'ME',
    'MD',
    'MA',
    'MI',
    'MN',
    'MS',
    'MO',
    'MT',
    'NE',
    'NV',
    'NH',
    'NJ',
    'NM',
    'NY',
    'NC',
    'ND',
    'OH',
    'OK',
    'OR',
    'PA',
    'RI',
    'SC',
    'SD',
    'TN',
    'TX',
    'UT',
    'VT',
    'VA',
    'WA',
    'WV',
    'WI', 
    'WY'
]

len(codes)

51

In [58]:
for state in codes:
    if state not in walkscores.state.to_list():
        print(state)

HI
ME
SD
VT
WV


In [59]:
for state in codes:
    if state not in rental.state.to_list():
        print(state)
        
# The positive here is that the missing states are consistent
# between walkscores and rental data, but future iterations
# could strive to include all US states

HI
ME
SD
VT
WV


In [110]:
# Inspect BLS Data - deal with city and state names

print(bls.shape)
bls.head()

(362, 5)


,city,state,occ_title,hourly_wage,annual_wage
0,Abilene,TX,Wellhead Pumpers,30.14,62680
1,Aguadilla-Isabela,PR,Telemarketers,8.16,16980
2,Akron,OH,"Patternmakers, Metal and Plastic",17.27,35930
3,Albany,OR,Fallers,35.98,74850
4,Albany-Schenectady-Troy,NY,"Physical Scientists, All Other",46.86,97470


In [111]:
bls.loc[bls.city.str.contains("Dallas")]

# Can simply go with first city in city names for creation
# of static endpoint. BLS endpoint itself is more robust
# so in ex. below, 'Fort Worth' can still be found. On
# `static` endpoint however, it would not be found.

# Considering that this `static` endpoint is only intended for use with
# the advanced search feature, this may be a workable approach ...

,city,state,occ_title,hourly_wage,annual_wage
83,Dallas-Fort Worth-Arlington,TX,Semiconductor Processing Technicians,17.86,37150


In [112]:
# Make copy
bls_slim = bls.copy()

In [113]:
bls_slim["city"] = [x.split("-")[0].strip() for x in bls_slim.city.to_list()]
bls_slim["state"] = [x.split("-")[0].strip() for x in bls_slim.state.to_list()]

In [114]:
print(bls_slim.shape)
bls_slim.head()

# This should be able to merge better with m2 ...

(362, 5)


,city,state,occ_title,hourly_wage,annual_wage
0,Abilene,TX,Wellhead Pumpers,30.14,62680
1,Aguadilla,PR,Telemarketers,8.16,16980
2,Akron,OH,"Patternmakers, Metal and Plastic",17.27,35930
3,Albany,OR,Fallers,35.98,74850
4,Albany,NY,"Physical Scientists, All Other",46.86,97470


## Perform Merges - BLS Last Due to Naming Conventions

In [30]:
# Rental + Walkscores

m1 = rental.merge(walkscores, how="inner", on=["city","state"])

print(m1.shape)
m1.head()

(444, 8)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore
0,King of Prussia,PA,1038,1232,1488,1862,2072,53.64
1,Las Cruces,NM,514,599,726,1045,1280,56.08
2,North East,MD,909,1079,1303,1630,1814,41.96
3,St. Cloud,MN,578,712,883,1218,1543,64.32
4,Pasadena,CA,1350,1606,2064,2806,3093,90.48


In [62]:
print("Merge 1 Unique States:", m1.state.nunique())
print("Merge 1 Unique Cities:", m1.city.nunique())

Merge 1 Unique States: 46
Merge 1 Unique Cities: 429


In [47]:
# Rental + Walkscores + Census

m2 = m1.merge(census, how="inner", on=["city","state"])

print(m2.shape)
m2.head()

(365, 9)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,popestimate2019
0,Las Cruces,NM,514,599,726,1045,1280,56.08,103432
1,St. Cloud,MN,578,712,883,1218,1543,64.32,68462
2,Crystal Lake,IL,1042,1209,1422,1819,2165,60.68,39829
3,Clarksville,TN,523,631,810,1109,1241,45.90,158146
4,Englewood,CO,1003,1225,1551,2253,2610,65.86,34917


In [85]:
print("Merge 2 Unique States:", m2.state.nunique())
print("Merge 2 Unique Cities:", m2.city.nunique())

Merge 2 Unique States: 46
Merge 2 Unique Cities: 365


In [119]:
# Rental + Walkscores + Census + BLS

m3 = m2.merge(bls_slim, how="inner", on=["city","state"])

m3 = m3.rename(columns={"popestimate2019":"population"})

print(m3.shape)
m3.head()

(135, 12)


,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,population,occ_title,hourly_wage,annual_wage
0,Las Cruces,NM,514,599,726,1045,1280,56.08,103432,Anthropologists and Archeologists,28.75,59800
1,St. Cloud,MN,578,712,883,1218,1543,64.32,68462,Ophthalmic Laboratory Technicians,15.31,31850
2,Clarksville,TN,523,631,810,1109,1241,45.90,158146,"Insulation Workers, Floor, Ceiling, and Wall",9.40,19560
3,Tyler,TX,659,733,898,1187,1297,56.72,106985,Helpers--Extraction Workers,20.89,43450
4,Fresno,CA,803,851,1063,1508,1763,67.66,531576,"Farmworkers and Laborers, Crop, Nursery, and G...",12.36,25710


In [120]:
print("Merge 3 Unique States:", m3.state.nunique())
print("Merge 3 Unique Cities:", m3.city.nunique())

Merge 3 Unique States: 45
Merge 3 Unique Cities: 135


In [117]:
for state in codes:
    if state not in m3.state.to_list():
        print(state)
        
# Idaho now added to list of states not included ...

# Still able to retain 45 states though which is encouraging. Again,
# future iterations will likely involve a concerted effort to retain more states
# for each feature

HI
ID
ME
SD
VT
WV


In [122]:
# Do some spot checks to ensure 'major' cities are still
# represented in m3

m3.loc[m3.city == "Seattle"]

# After spot checking several "major" cities, I am satisfied
# with the retention, with the only concerning absence (out
# of about 20 experiments) being "Newark"


### UPDATE: "Seattle" is now a missing city as well due to not
# having wage data, but in consideration of the advanced search,
# it will be important for that data to be clean and void of
# missing information

,city,state,studio,onebr,twobr,threebr,fourbr,walkscore,population,occ_title,hourly_wage,annual_wage


In [121]:
### Export m3

PATH = "/Users/ekselan/Desktop/LAMBDA/LABS/Labs26-Citrics-DS-TeamA/data/static/static.csv"

m3.to_csv(PATH, index=False)